# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [12]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
# from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [38]:

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "deepseek-r1"

In [14]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
api_key = os.getenv('OPENAI_API_KEY')

In [15]:
# set up environment


system_message = "You are a very helpful expert, you take very complex questions or ideas and explain them in very clear and easy to understand manner"


In [16]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [31]:
# Get gpt-4o-mini to answer, with streaming
def expertly():
    openai = OpenAI()
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": question}
      ],
      stream = True,
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [32]:
expertly()

Let's break down the code you provided to understand what it does and why.

### Code Breakdown

1. **`{...}`**: This denotes a set comprehension in Python. A set is a collection of unique items. The code inside the braces will generate a set based on the conditions provided.

2. **`for book in books`**: This part loops through each item in the `books` collection (which is presumably a list of dictionaries or similar structures, where each item represents a book).

3. **`book.get("author")`**: For each `book`, it tries to retrieve the value associated with the key `"author"`. The `get` method is used instead of directly accessing the dictionary with `book["author"]` to avoid KeyErrors if the key does not exist.

4. **`if book.get("author")`**: This conditional checks if the value retrieved is truthy (i.e., not `None`, empty string, etc.). If it’s truthy, the code proceeds to include it in the set.

5. **`yield from`**: This is a special statement in Python that is typically used within a generator function. It yields all values from an iterable. In this case, it is yielding all the values from the set generated by the set comprehension.

### Purpose of the Code

So, when you put it all together, this line of code does the following:

- It creates a set of unique authors from a list of `books`, where each book is expected to have an `"author"` key. 
- It only includes the authors that actually exist (i.e., that are not `None` or empty).
- Finally, it yields each author one by one, allowing you to iterate through them in the context of a generator.

### Practical Example

Suppose you have the following list of books:

python
books = [
    {"title": "Book1", "author": "Author1"},
    {"title": "Book2", "author": "Author2"},
    {"title": "Book3"},  # No author
    {"title": "Book4", "author": "Author1"},  # Duplicate author
]


If you execute the code you provided, the result would be:

- It will create a set `{"Author1", "Author2"}` because `"Author1"` appears twice but sets only keep unique items.
- When you use `yield from`, you will be able to iterate through these authors in a way that provides them one at a time.

### Summary

To summarize, this code efficiently collects all unique authors from a list of books (ignoring any entries without authors) and allows you to yield them for further processing in a clean manner. It's a good example of combining set comprehension with generator functionality in Python.

In [ ]:
# import ollama
# messages=[
#             {"role": "system", "content": system_message},
#             {"role": "user", "content": question}
#       ]

# response = ollama.chat(model=MODEL, messages=messages)
# print(response['message']['content'])

In [41]:
# Get Llama 3.2 to answer
def expertllama():
    ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    openai = OpenAI()
    stream = ollama.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": question}
      ],
      stream = True,
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [42]:
expertllama()

**Diving into the Code**

This line of code is a part of a Python generator expression, which is used to generate a sequence of values. Let's break it down:

### Yield From Clause

The `yield from` clause allows you to yield values from another iterable or generator.

### Generator Expression

A generator expression is similar to a list comprehension, but instead of creating an entire list in memory, it generates the values on-the-fly as needed.

**Let's Break Down the Code**

Here's a step-by-step explanation:

1. `{book.get("author") for book in books if book.get("author")}`

   - This is a generator expression that:
     - Tries to get the value for `"author"` from each `book` (assuming `books` is an iterable).
     - Includes only the values where the attribute exists (`if book.get("author"`), meaning the `get` method returns a truthy value, not `None` or another falsy one).
   
2. `yield from {}`

   - This clause tells Python to "yield" all the generated items (from the generator expression in step 1) and essentially wraps them up into another new iterable object. However, here's the key difference: it still needs an actual iterable as an argument.

However, there is a small catch! Here `{book.get("author") for book in books if book.get("author")}` returns a **new** iterable (which might be empty!), if we try to "yield" from it directly in the line `yield from {...}`, Python expects another actual iterable or generator.

So, what happens next is this:

Because "{book.get('author') for book in books if book.get('author')} becomes an actual iterable, yielding over it means that yielded value becomes a single item returned by each call to __next__() and will repeat the values indefinitely until exhausted. As result the whole `yield from { book.get("author") for book in books if book.get("author"}} generates and yields **all** available authors.

### Example

Without `yield from` we would have:

python
for author in "{book.get('author') for book in books}" :

    yield author


But this is an explicit loop with a generator expression which can get more error-prone because it can fail if the books dictionary doesn't contain authors because then iteration would happen only once.